In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v1

In [2]:
gamma = 0.9   #折扣因子，越接近0越近视
rows = 5      #记得行数和列数这里要同步改
columns = 5
# gridworld = GridWorld_v1.GridWorld_v1(rows=rows, columns=columns, forbiddenAreaNums=4, targetNums=2, seed = random.randint(1,1000))
# gridworld = GridWorld_v1.GridWorld_v1(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v1.GridWorld_v1(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v1.GridWorld_v1(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓
policy = np.argmax(qtable,axis=1)    #初始策略
gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
⬆️⬆️⬆️⬆️⬆️
⬆️⏫️⏫️⬆️⬆️
⬆️⬆️⏫️⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️


In [3]:
policy = np.random.randint(0,5,size=(rows*columns)) 
#随机[0,5)的整数，代表策略
#这里其实不严谨，因为策略是可以不百分百选一个方向的，可以0.5向上，0.5向右，诸如此类。
#但先不考虑那种情况，因为画图不好画，代码实现逻辑是没差多少的

In [8]:
#求解贝尔曼方程
gridworld.show()                     #打印gridworld
gridworld.showPolicy(policy)         #打印策略
print("random policy")
#policy evaluation
value = np.zeros(rows*columns)
value_pre = value.copy()+1

cnt = 0
while(np.sum((value_pre-value)**2)>0.001): # while vk has not converged
    #policy evaluation
    value_pre = value.copy() #用来验证整个迭代是否收敛的

    value0 = value.copy()+1  #这里是随机一个值，然后通过迭代的方式求解贝尔曼方程
                             #这里写了固定，也可以随机，最终都会收敛到同一个结果
    
    truncatedCnt = 10       # 1:迭代50次  2：迭代26次 3：迭代18次 4：迭代14次  10：迭代6次 100：迭代2次
    while(np.sum((value0-value)**2)>0.001): # vkj has not converged
        value0 = value.copy()
        
        truncatedCnt = truncatedCnt-1  #这里这里加个限制，其实就是truncated policy iteration了
        if truncatedCnt<0:             #其实就是贝尔曼迭代次数
            break
                
        for i in range(rows * columns):   #使用当前策略policy，计算每个state的value，进行迭代
            j = policy[i]                 #不用遍历5个action了，直接百分百选择policy的策略
            score, nextState = gridworld.getScore(i,j)   #返回得分以及下一步的state id
            value[i] = score + value0[nextState] * gamma #贝尔曼迭代
    
    # policy improvement
    for i in range(rows * columns):
        for j in range(5): # 5个action
            score,nextState = gridworld.getScore(i,j)        #获取Si状态中，执行动作j后的（得分，下一个状态）
            qtable[i][j] = score + gamma * value[nextState]   

    policy = np.argmax(qtable,axis=1)  #更新策略，非常无敌
    
    gridworld.showPolicy(policy)      #各种打印信息
    print(value.reshape(rows,columns))
    cnt = cnt+1
    print(cnt)

    

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
random policy
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[0.         0.38742049 0.8178877  1.2961846  1.8276256 ]
 [0.         0.         1.2961846  1.8276256  2.4181156 ]
 [0.         0.         6.5132156  2.4181156  3.0742156 ]
 [0.         6.5132156  6.5132156  6.5132156  3.8032156 ]
 [0.         5.5132156  6.5132156  5.5132156  4.6132156 ]]
1
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[2.27101786 2.65843834 3.08890555 3.56720245 4.09864345]
 [1.92233941 2.27101786 3.56720245 4.09864345 4.68913345]
 [1.60852882 1.32609928 8.78423345 4.68913345 5.34523345]
 [1.32609928 8.78423345 8.78423345 8.78423345 6.07423345]
 [1.0719127  7.78423345 8.78423345 7.78423345 6.88423345]]
2
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[3.06287282 3.45029331 3.88076052 4.35905742 4.89049842]
 [2.71419438 3.06287282 4.35905742 4.89049842 5.48098842]
 [2.40038378 2.11795425 9.57

In [9]:
#求解贝尔曼方程
gridworld.show()                     #打印gridworld
gridworld.showPolicy(policy)         #打印策略
print("random policy")
#policy evaluation
value = np.zeros(rows*columns)
value_pre = value.copy()+1

cnt = 0
while(np.sum((value_pre-value)**2)>0.001):
    #policy evaluation
    value_pre = value.copy() #用来验证整个迭代是否收敛的
    value0 = value.copy()+1  #这里是随机一个值，然后通过迭代的方式求解贝尔曼方程
                             #这里写了固定，也可以随机，最终都会收敛到同一个结果
    truncatedCnt = 10       # 1:迭代50次  2：迭代26次 3：迭代18次 4：迭代14次  10：迭代6次 100：迭代2次
    while(np.sum((value0-value)**2)>0.001):
        value0 = value.copy()
        
        truncatedCnt = truncatedCnt-1  #这里这里加个限制，其实就是truncated policy iteration了
        if truncatedCnt<0:             #其实就是贝尔曼迭代次数
            break
                
        for i in range(rows * columns):   #使用当前策略policy，计算每个state的value，进行迭代
            j = policy[i]                 #不用遍历5个action了，直接百分百选择policy的策略
            score, nextState = gridworld.getScore(i,j)   #返回得分以及下一步的state id
            value[i] = score + value0[nextState] * gamma #贝尔曼迭代
    

    #policy improvement
    for i in range(rows * columns):
        for j in range(5): # 5个action
            score,nextState = gridworld.getScore(i,j)     #获取Si状态中，执行动作j后的（得分，下一个状态）
            qtable[i][j] = score + gamma * value[nextState]

    policy = np.argmax(qtable,axis=1)  #更新策略，非常无敌

    
    gridworld.showPolicy(policy)      #各种打印信息
    print(value.reshape(rows,columns))
    cnt = cnt+1
    print(cnt)

    

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
random policy
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[0.         0.38742049 0.8178877  1.2961846  1.8276256 ]
 [0.         0.         1.2961846  1.8276256  2.4181156 ]
 [0.         0.         6.5132156  2.4181156  3.0742156 ]
 [0.         6.5132156  6.5132156  6.5132156  3.8032156 ]
 [0.         5.5132156  6.5132156  5.5132156  4.6132156 ]]
1
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[2.27101786 2.65843834 3.08890555 3.56720245 4.09864345]
 [1.92233941 2.27101786 3.56720245 4.09864345 4.68913345]
 [1.60852882 1.32609928 8.78423345 4.68913345 5.34523345]
 [1.32609928 8.78423345 8.78423345 8.78423345 6.07423345]
 [1.0719127  7.78423345 8.78423345 7.78423345 6.88423345]]
2
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[3.06287282 3.45029331 3.88076052 4.35905742 4.89049842]
 [2.71419438 3.06287282 4.35905742 4.89049842 5.48098842]
 [2.40038378 2.11795425 9.57